<a href="https://colab.research.google.com/github/uttam112/sprinklr/blob/main/Text_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading the Urls from txt file

In [ ]:
urls = []
with open('SprinklrDotCom.txt', 'r') as file:
    for line in file:
        if(line.strip() not in urls):
            urls.append(line.strip())

print(len(urls))

#Extacting Text From the Urls

In [ ]:
!pip install langchain
!pip install unstructured

In [ ]:
#removing texts that are present in header and footers

def filter_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub('([ ]*\n)+', '\n', text)
    unwanted = ["Sprinklr\n","Request Demo\n","English (US)\n","English (GB)\n","English (AU)\n","English (SG)\n","Fran ais\n","Portugu s\n","Deutsch\n","Espa ol\n","Login\n"]
    unwanted = unwanted + ["Privacy\n","Do Not Sell or Share My Info\n","Cookie Preferences\n","Modern Slavery Statement\n","Index Egalit \n","Terms","Loading...\n","Connect with us\n"]
    unwanted = unwanted + ["Platform\n","Services\n","Integrated Services\n","Training\n","Partners\n","Agency\n","Developer Portal\n","Company\n","Our Story\n","Newsroom\n","Investor Relations\n","Culture & Talent\n","Careers\n","Contact Us\n","Resources\n","Analyst Reports\n","Product Demo Days\n","eBooks\n","Events\n","Blog (US)\n","Customer Stories\n"]
    text = re.sub('|'.join(map(re.escape, unwanted)), '', text)

    return text

In [ ]:
#list to store the extrated text with source
datas = []

In [ ]:
#Extacting text along with the source url
loaders = UnstructuredURLLoader(urls=[urls[0]])
data = loaders.load()
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser",from_encoding="iso-8859-1")
    text = soup.get_text(separator='\n') #text present in the url
    text = filter_text(text)
    # data = loaders.load()
    data[0].page_content = text
    data[0].metadata["source"] = url
    datas.append(data[0])
    i+=1
    if(i%100==0):
        print(i)


In [ ]:
import re
import requests
from bs4 import BeautifulSoup

tot_len = len(urls)
sz = len(urls)/100 #Make a separate txt file for each 100 url


for i in range(0,sz):
    j=0
    print(i)
    data = []
    for url in urls[i*100:min(tot_len,(i+1)*100)]:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser",from_encoding="iso-8859-1")
        text = soup.get_text() #text present in the url
        text = re.sub(r'[^\x00-\x7F]+',' ', text) #remove non-ascii characters
        data.append(text)
        j+=1
    filename = "new_context/context"+str(i)
    with open(filename, 'w') as file:
        for item in data:
            item = re.sub('\n+', '\n', item)
            file.write(str(item) + '\n')
